In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hurtos-medelln/hurto_a_persona_medata.csv
/kaggle/input/zonas-tursticas-medelln/atractivos_turisticos.csv


In [2]:
df_lugares = pd.read_csv('/kaggle/input/zonas-tursticas-medelln/atractivos_turisticos.csv', encoding='latin1')
# Convertir las columnas de coordenadas a tipo float
df_lugares['x_origen_nacional'] = df_lugares['x_origen_nacional'].astype(float)
df_lugares['y_origen_nacional'] = df_lugares['y_origen_nacional'].astype(float)
df_lugares.head()

,OBJECTID,Shape,imperdible,cod_comuna,cod,nombre_sitio,tipo_atractivo,orden_visualizacion,direccion,sitio_web,nombre_comuna,cod_barrio,barrio,x_origen_nacional,y_origen_nacional,latitud,longitud
0,1,"(4713317.744900001, 2246254.5177000016)",NO,15,AT001,Aeroparque Parque Juan Pablo II,Parque temático y recreativo,14,CR 70 No. 16 - 04,https://aeroparquejuanpablo.gov.co/,Guayabal,Inst_19,Parque Juan Pablo II,4.713318e+06,2.246255e+06,6.222290,-75.591675
1,2,"(4713559.109200001, 2245875.2638000026)",NO,15,AT002,Aeropuerto Enrique Olaya Herrera,Aeropuerto de pasajeros,37,CR 65 A No. 13 - 157,https://www.aeropuertomedellin.co/,Guayabal,Inst_19,Parque Juan Pablo II,4.713559e+06,2.245875e+06,6.218872,-75.589478
2,3,"(4716641.4362, 2250356.483500004)",NO,10,AT003,Barrio Prado,Sector urbano o rural de interés,0,CR 50 No. 62-99,,La Candelaria,1001,Prado,4.716641e+06,2.250356e+06,6.259521,-75.561829
3,4,"(4716008.103700001, 2249319.9003000017)",NO,10,AT004,Basílica Menor Nuestra Señora de la Candelaria,Basílica,35,CR 49 a No. 50 - 85,https://arqmedellin.co/parroquias/nuestra-seno...,La Candelaria,1019,La Candelaria,4.716008e+06,2.249320e+06,6.250122,-75.567505
4,5,"(4713705.0748000005, 2249459.4629000034)",NO,11,AT005,Bulevar Libertadores de América,Sector urbano o rural de interés,2,CR 70 entre la CL 47D y circular 1A,,Laureles Estadio,1117,Florida Nueva,4.713705e+06,2.249459e+06,6.251282,-75.588318


In [3]:
import pandas as pd

# Cargar el archivo CSV
# Reemplaza 'ruta/del/archivo.csv' con la ruta de tu archivo
df_hurtos = pd.read_csv('/kaggle/input/hurtos-medelln/hurto_a_persona_medata.csv', encoding='latin1', delimiter=';')

# Asegurarse de que la columna de fecha esté en formato de fecha (si no, convertirla)
df_hurtos['fecha_hecho'] = pd.to_datetime(df_hurtos['fecha_hecho'], errors='coerce')

# 1. Eliminar filas sin fecha (NaT en la columna de fecha)
df_hurtos = df_hurtos.dropna(subset=['fecha_hecho'])

# 2. Filtrar y eliminar datos hasta el año 2015
df_hurtos = df_hurtos[df_hurtos['fecha_hecho'].dt.year > 2015]

# 3. Eliminar las columnas no deseadas
columnas_a_eliminar = [
    'grupo_actor', 'actividad_delictiva', 'parentesco', 
    'ocupacion', 'discapacidad', 'grupo_especial', 'estado_civil'
]
df_hurtos = df_hurtos.drop(columns=columnas_a_eliminar)
df_hurtos = df_hurtos.dropna(axis=1, how='all')

# Reemplazar valores -1 en la columna 'edad' por NaN
df_hurtos['edad'] = df_hurtos['edad'].replace(-1, np.nan)

# Reemplazar comas por puntos en las columnas de latitud y longitud
df_hurtos['latitud'] = df_hurtos['latitud'].str.replace(',', '.', regex=False)
df_hurtos['longitud'] = df_hurtos['longitud'].str.replace(',', '.', regex=False)

# Convertir las columnas de latitud y longitud a tipo numérico
df_hurtos['latitud'] = pd.to_numeric(df_hurtos['latitud'], errors='coerce')
df_hurtos['longitud'] = pd.to_numeric(df_hurtos['longitud'], errors='coerce')

# Definir límites de latitud y longitud para el Área Metropolitana de Medellín
lat_min, lat_max = 6.0, 6.5
lon_min, lon_max = -75.6, -75.3

# Filtrar los lugares turísticos y los incidentes para incluir solo los puntos dentro de los límites

df_hurtos = df_hurtos[(df_hurtos['latitud'] >= lat_min) & (df_hurtos['latitud'] <= lat_max) &
                              (df_hurtos['longitud'] >= lon_min) & (df_hurtos['longitud'] <= lon_max)]
df_lugares = df_lugares[(df_lugares['latitud'] >= lat_min) & (df_lugares['latitud'] <= lat_max) &
                              (df_lugares['longitud'] >= lon_min) & (df_lugares['longitud'] <= lon_max)]
# Mostrar las primeras filas para verificar los cambios
df_hurtos.head()

,fecha_hecho,latitud,longitud,caso,valor,cantidad,nombre_barrio,codigo_barrio,codigo_comuna,lugar,...,edad,medio_transporte,conducta,modalidad,conducta_especial,arma_medio,bien,categoria_bien,grupo_bien,color
0,2017-01-01 13:00:00,6.273576,-75.553778,VERDADERO,550000,1,Campo Valdés No.2,#0303,3,Vía pública,...,64.0,Caminata,Hurto a persona,Atraco,De celular,Objeto contundente,Celular,Tecnología,Mercancía,Sin dato
1,2017-02-01 06:00:00,6.220378,-75.582590,VERDADERO,2000000,1,Santa Fé,#1504,15,Vehículo particular,...,32.0,Caminata,Hurto a persona,Rompimiento de ventana,No,No,Sin dato electrodomésticos,Electrodomésticos,Mercancía,Sin dato
2,2017-11-01 11:00:00,6.250438,-75.568245,FALSO,60000,1,La Candelaria,#1019,10,Vía pública,...,32.0,Caminata,Hurto a persona,Atraco,De celular,Arma cortopunzante,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,Sin dato
3,2017-11-01 11:30:00,6.250113,-75.564120,FALSO,2000000,1,La Candelaria,#1019,10,Vía pública,...,51.0,Caminata,Hurto a persona,Atraco,No,Arma cortopunzante,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,Sin dato
197,2017-01-02 16:30:00,6.290393,-75.564681,FALSO,0,1,Tricentenario,#0510,5,Estación del Metro,...,39.0,Caminata,Hurto a persona,Cosquilleo,Sin dato,No,Cédula,Documentos,Mercancía,Amarillo


In [4]:
df_hurtos.head()

,fecha_hecho,latitud,longitud,caso,valor,cantidad,nombre_barrio,codigo_barrio,codigo_comuna,lugar,...,edad,medio_transporte,conducta,modalidad,conducta_especial,arma_medio,bien,categoria_bien,grupo_bien,color
0,2017-01-01 13:00:00,6.273576,-75.553778,VERDADERO,550000,1,Campo Valdés No.2,#0303,3,Vía pública,...,64.0,Caminata,Hurto a persona,Atraco,De celular,Objeto contundente,Celular,Tecnología,Mercancía,Sin dato
1,2017-02-01 06:00:00,6.220378,-75.582590,VERDADERO,2000000,1,Santa Fé,#1504,15,Vehículo particular,...,32.0,Caminata,Hurto a persona,Rompimiento de ventana,No,No,Sin dato electrodomésticos,Electrodomésticos,Mercancía,Sin dato
2,2017-11-01 11:00:00,6.250438,-75.568245,FALSO,60000,1,La Candelaria,#1019,10,Vía pública,...,32.0,Caminata,Hurto a persona,Atraco,De celular,Arma cortopunzante,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,Sin dato
3,2017-11-01 11:30:00,6.250113,-75.564120,FALSO,2000000,1,La Candelaria,#1019,10,Vía pública,...,51.0,Caminata,Hurto a persona,Atraco,No,Arma cortopunzante,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,Sin dato
197,2017-01-02 16:30:00,6.290393,-75.564681,FALSO,0,1,Tricentenario,#0510,5,Estación del Metro,...,39.0,Caminata,Hurto a persona,Cosquilleo,Sin dato,No,Cédula,Documentos,Mercancía,Amarillo


In [5]:
df_hurtos.describe()

,fecha_hecho,latitud,longitud,cantidad,edad
count,81518,81518.000000,81518.000000,81518.0,80653.000000
mean,2019-10-02 17:11:14.287887616,6.247547,-75.572511,1.0,34.657049
min,2016-01-01 06:18:00,6.066655,-75.599995,1.0,1.000000
25%,2018-04-11 20:55:00,6.234387,-75.582317,1.0,25.000000
50%,2019-09-07 21:00:00,6.248920,-75.570349,1.0,32.000000
75%,2021-06-02 00:30:00,6.260560,-75.565150,1.0,41.000000
max,2022-12-12 23:30:00,6.372461,-75.337630,1.0,94.000000
std,NaN,0.025089,0.013875,0.0,12.351821


In [6]:
df_hurtos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81518 entries, 0 to 350884
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha_hecho        81518 non-null  datetime64[ns]
 1   latitud            81518 non-null  float64       
 2   longitud           81518 non-null  float64       
 3   caso               81518 non-null  object        
 4   valor              81518 non-null  object        
 5   cantidad           81518 non-null  int64         
 6   nombre_barrio      81518 non-null  object        
 7   codigo_barrio      81518 non-null  object        
 8   codigo_comuna      81518 non-null  object        
 9   lugar              81518 non-null  object        
 10  sede_receptora     81518 non-null  object        
 11  sexo               81518 non-null  object        
 12  edad               80653 non-null  float64       
 13  medio_transporte   81518 non-null  object        
 14  conducta  

# Cálculos

In [7]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic

# Definir la función de Haversine vectorizada
def haversine_vector(lat1, lon1, lat2, lon2):
    # Convertir a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Fórmula de Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371  # Radio de la Tierra en km
    return R * c * 1000  # Distancia en metros

# Vectorizamos las columnas latitud y longitud
hurtos_lat = df_hurtos['latitud'].values
hurtos_lon = df_hurtos['longitud'].values
lugares_lat = df_lugares['latitud'].values
lugares_lon = df_lugares['longitud'].values

# Calculamos la distancia entre todos los hurtos y lugares turísticos
distancias = np.array([
    haversine_vector(hurtos_lat, hurtos_lon, lat, lon)
    for lat, lon in zip(lugares_lat, lugares_lon)
]).T

# Establecemos un radio de proximidad en metros (por ejemplo, 500 metros)
radio_proximidad = 500

# Creamos una matriz booleana para las distancias cercanas
distancias_cercanas = distancias <= radio_proximidad

# Agrupar incidentes cercanos por lugar turístico
# Usamos np.where para obtener índices de hurtos cercanos a lugares turísticos
incidentes_por_lugar = {}

# Iteramos por los índices donde hay True en la matriz de distancias_cercanas
for hurto_idx, lugar_idx in zip(*np.where(distancias_cercanas)):
    lugar_id = df_lugares.iloc[lugar_idx]['cod']
    if lugar_id not in incidentes_por_lugar:
        incidentes_por_lugar[lugar_id] = 0
    incidentes_por_lugar[lugar_id] += 1

# Convertimos la información en un DataFrame
incidentes_df_lugares = pd.DataFrame(list(incidentes_por_lugar.items()), columns=['Lugar_Turistico', 'Numero_Incidentes'])

incidentes_df_lugares


,Lugar_Turistico,Numero_Incidentes
0,AT046,736
1,AT069,603
2,AT004,11805
3,AT007,8141
4,AT013,10826
...,...,...
82,AT055,213
83,AT051,125
84,AT019,33
85,AT016,8


In [8]:
import folium
from folium.plugins import HeatMap
import pandas as pd

# Unir los DataFrames por el código del lugar turístico
df_merged = pd.merge(df_lugares, incidentes_df_lugares, left_on='cod', right_on='Lugar_Turistico')

# Crear el mapa centrado en el área metropolitana de Antioquia
mapa = folium.Map(location=[6.2442, -75.5736], zoom_start=12)

# Crear una lista de ubicaciones con intensidad basada en el número de incidentes
heat_data = []
for idx, lugar in df_merged.iterrows():
    lat = lugar['latitud']
    lon = lugar['longitud']
    cantidad_incidentes = lugar['Numero_Incidentes']
    
    # Agregar la ubicación con un peso proporcional a la cantidad de incidentes
    heat_data.append([lat, lon, cantidad_incidentes])

# Crear el mapa de calor
HeatMap(heat_data).add_to(mapa)

# Mostrar el mapa
mapa

In [9]:
import pandas as pd

# Paso 1: Obtener el número máximo de incidentes
max_incidentes = incidentes_df_lugares['Numero_Incidentes'].max()

# Paso 2: Calcular el ISAT (Índice de Seguridad de Atractivos Turísticos)
incidentes_df_lugares['ISAT'] = incidentes_df_lugares['Numero_Incidentes'] / max_incidentes

# Paso 3: Visualizar el ISAT
incidentes_df_lugares[['Lugar_Turistico', 'Numero_Incidentes', 'ISAT']]


,Lugar_Turistico,Numero_Incidentes,ISAT
0,AT046,736,0.062000
1,AT069,603,0.050796
2,AT004,11805,0.994440
3,AT007,8141,0.685789
4,AT013,10826,0.911970
...,...,...,...
82,AT055,213,0.017943
83,AT051,125,0.010530
84,AT019,33,0.002780
85,AT016,8,0.000674


In [10]:
incidentes_df_lugares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Lugar_Turistico    87 non-null     object 
 1   Numero_Incidentes  87 non-null     int64  
 2   ISAT               87 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ KB
